In [5]:
import pickle
import pandas as pd
from schemas import DiabetesInput

In [6]:
with open('/content/voting_classififer.pkl','rb') as f:
  model=pickle.load(f)

In [7]:
with open('/content/scaler.pkl','rb') as f:
  scaler=pickle.load(f)

In [8]:
with open('/content/feature_columns.pkl','rb') as f:
  feature_columns=pickle.load(f)

In [9]:
with open("imputer.pkl", "rb") as f:
    imputer = pickle.load(f)

In [10]:
from fastapi import FastAPI
app=FastAPI(
    title="Diabetes Prediction API",
    description="API for predicting diabetes using an ensemble model",
    version="1.0.0",
)

In [11]:
@app.get("/")
def home():
  return {"message":"Diabetes Prediction API is running"}

In [12]:
@app.post("/predict")
def predict_diabetes(user_input: DiabetesInput):
  input_list=user_input.to_model_input()
  input_df=pd.DataFrame([input_list],columns=feature_columns)
  input_imputed=imputer.transform(input_df)
  input_scaled=scaler.transform(input_imputed)

  prediction=model.predict(input_scaled)[0]
  probability=model.predict_proba(input_scaled)[0][1]

  if prediction==1:
    message="High risk of diabetes"
  else:
    message="Low risk of diabetes"

  return{
      "prediction":int(prediction),
      "probability":round(probability,2),
      "message":message
  }

